In [8]:
from optimum.onnxruntime import ORTModelForQuestionAnswering, ORTModelForAudioClassification, ORTModelForSequenceClassification, ORTModelForSpeechSeq2Seq
from transformers import AutoConfig, Wav2Vec2Processor, AutoTokenizer, PretrainedConfig, WhisperProcessor
import time
import librosa
import numpy as np
import pandas as pd
import os

import audio_base 
import metric
import config

# Речь в текст

In [9]:

sampling_rate =  16000
path_a ='01_happiness_anger a_020.wav'
speech, sr = librosa.load(path_a, sr=sampling_rate)
model_text = audio_base.SpeechtoText('whisper-tiny_onnx', "openai/whisper-base", "openai/whisper-tiny")
model_text.run(speech)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using a model of type whisper to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


[' Слушай, я потратил обретьом кучу денег для того, чтобы притоситься в эту дару. Это что вообще такое? Посмотри на официантов, они все в черных каких-то рубашках с кислыми минами. Даже никто из них до сих пор не подошел к нам.']

In [10]:
path_a ='01_happiness_anger a_020.wav'
speech, sr = librosa.load(path_a, sr=sampling_rate)
model_em = audio_base.SpeechtoEmotion("wav2vec2-xls-r-300m-emotion-ru_onnx", "KELONMYOSA/wav2vec2-xls-r-300m-emotion-ru")
model_em.run(speech)

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


'angry'

# Текст в эмоции

In [11]:
model_textem = audio_base.TexttoEmotion("rubert-tiny2-russian-emotion-detection_onnx", "Djacon/rubert-tiny2-russian-emotion-detection")
model_textem.run("ой беда, какая беда")

'sadness'

## Общий поток распознавания

In [12]:
path_a ='01_happiness_anger a_020.wav'
# path_a = '1725.wav'


# создаем модели

model_text = audio_base.SpeechtoText(config.model_name_speech_to_text[0], config.model_name_speech_to_text[1], config.model_name_speech_to_text[2])
model_textem = audio_base.TexttoEmotion(config.model_name_text_to_emo[0], config.model_name_text_to_emo[1])
model_em = audio_base.SpeechtoEmotion(config.model_name_speech_to_emo[0],config.model_name_speech_to_emo[1],)

# длина эпизода записи в с. 
dt = 1
d = 16000 * dt
n_d = 4


# rez_text_all = model_text.run(speech)
# rez_text_all_buf = rez_text_all[0] 
# rez_text_all_list = rez_text_all[0].split(' ')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using a model of type whisper to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [13]:
    
s, df =  audio_base.check_model(path_a, dt=1, d=d, n_d=4,  model_text=model_text, model_textem=model_textem, model_em=model_em)
s, df.head()

(' Слушай, потратил об этом  кучу денег для  того, что  притащится в эту дару. Это что   вообще такое? Посмотри на офицантов. Они все в    черных каких-то рубашках с   кислыми минами. Даже   никто из них до сих пор не  подошел к нам.',
      t                                         sub_string     word  \
 0  0.0   Слушай, потратил об этом кучу денег. Для того...  Слушай,   
 1  0.5   кучу денег для того, чтобы притащиться в эту ...     кучу   
 2  1.0   Для того, что притащится в эту дару, это что ...      Для   
 3  1.5   что притащится в эту дару. Это что вообще так...      что   
 4  2.0   В эту дару это что вообще такое посовинайся ц...        В   
 
           em1    em2  emotion  k2  k1  
 0     neutral  angry     0.50  26   0  
 1     neutral  angry     0.50  16   0  
 2     neutral  angry     0.50  15   4  
 3  enthusiasm  angry     0.55  36   4  
 4  enthusiasm  angry     0.55  20  19  )

In [14]:
s

' Слушай, потратил об этом  кучу денег для  того, что  притащится в эту дару. Это что   вообще такое? Посмотри на офицантов. Они все в    черных каких-то рубашках с   кислыми минами. Даже   никто из них до сих пор не  подошел к нам.'

In [15]:
dir_rez = 'save_rez/' # создать катол
df.to_csv(dir_rez + '.'.join(path_a.split('/')[-1].split('.')[:-1]) + '.csv')